# Example of using the orm classes

Imports

In [1]:
from os import remove
#from utils.io import striplog_from_text
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

In [3]:
remove('./tmp_files/test_orm_db.db')

Create the db engine and the database structure

In [4]:
engine = create_engine('sqlite:///tmp_files/test_orm_db.db', echo=True)

In [5]:
Base.metadata.create_all(engine)

2021-07-01 18:25:33,366 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-01 18:25:33,367 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-07-01 18:25:33,368 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-01 18:25:33,370 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-07-01 18:25:33,371 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-01 18:25:33,372 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Positions")
2021-07-01 18:25:33,372 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-01 18:25:33,373 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Positions")
2021-07-01 18:25:33,373 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-01 18:25:33,374 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("IntervalData")
2021-07-01 18:25:33,375 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-01 18:25:33,375 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("IntervalData")
2021-07-01 18:25:33,376 INFO sqlalchemy.engine.E

### Create the objects from las files

In [6]:
lexicon = Lexicon.default()

In [7]:
borehole_dict = {'F01':ROOT_DIR+'/data/test.las', 'F02':ROOT_DIR+'/data/test.csv'} # boreholes to insert into the db

In [8]:
int_id = 0
bh_id = 0
pos_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
pos_dict = {}
x = [0., 20.]
y = [0., 120.]

for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(BoreholeOrm(id=bh))
    
    strip=striplog_from_text(filename)
    
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        top = PositionOrm(id=pos_id, upper=interval.top.upper, 
                          middle=interval.top.middle,  
                          lower=interval.top.lower, 
                          x=x[bh_id], y=y[bh_id])
        
        base = PositionOrm(id=pos_id+1, upper=interval.base.upper, 
                           middle=interval.base.middle,  
                           lower=interval.base.lower, 
                           x=x[bh_id], y=y[bh_id])
        
        d.update({int_id:{'description':interval.description, 
                          'interval_number' : interval_number, 
                          'top': top, 'base': base 
                         }})
        
        interval_number+=1
        int_id += 1
        pos_id += 2

 

    print(d)
    boreholes[bh_id].intervals_values = d
    #boreholes[bh_id].components_values = c
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}

<ipython-input-8-568f20703c09>:14: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  boreholes.append(BoreholeOrm(id=bh))
<ipython-input-8-568f20703c09>:14: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.int_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Intervals.id to Linkintervalcomponent.int_id

NameError: name 'striplog_from_text' is not defined

Create the session

In [9]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [10]:
p = Project(session)
p.add_components(components)

2021-07-01 18:26:36,804 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-01 18:26:36,806 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-07-01 18:26:36,807 INFO sqlalchemy.engine.Engine [generated in 0.00045s] ()


AttributeError: 'list' object has no attribute 'keys'

Add boreholes into the database

In [ ]:
for bh in boreholes:
    p.add_borehole(bh)

In [ ]:
p.boreholes[0].id

In [ ]:
p.commit()

In [ ]:
p.boreholes[0].intervals[0].description

In [ ]:
session.close()

In [ ]:
p.plot3d(x3d=True)